# HW2 Zhiyu Xu
You will build a simple n-gram language model that can be used to generate
random text resembling a source document.

### Part 1
The first part is to import the packages we need and also get the source text from the given url.

In [1]:
import requests
import random, re
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

head = {
    'User-Agent':'Mozilla/4.0(compatible;MSIE 5.5;Windows NT)',
}

url = 'https://storm.cis.fordham.edu/~yli/data/MyShakespeare.txt'
html = requests.get(url, headers = head)
s = html.text   
print(s)

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



### Part 2
The NGram class.

P.S. I have no idea why you put the unknown word hint in the function 'e' part, because I think all those four functions are length and count, which has nothing to do with the probability. This does bother me for a while. I treat the results of first two len function as length and last two frequency ones as count.

In [93]:
import nltk

class NGram:

    
    def _init_(self, n, text):          
        head = {
            'User-Agent':'Mozilla/4.0(compatible;MSIE 5.5;Windows NT)',
        }
        
        url = 'https://storm.cis.fordham.edu/~yli/data/MyShakespeare.txt'
        html = requests.get(url, headers = head)
        s = html.text         
        
        self.n = n
        self.text = s
        
    def update(self, text):
        # The input text is padded with ‘.’ as the prefix, according to the requirement.
        # The result is the ngram for the text.
        if self.n >= 2:
            text = '. '*(self.n-1) + text
            
        tokens = word_tokenize(text)
        ngrams = zip(*[tokens[i:] for i in range(self.n)])
        result = [" ".join(ngram) for ngram in ngrams]
        return result
    
    def get_vocab(self):
        tokens = word_tokenize(s)
        return set(tokens)
    
    def size_vocab(self):
        size = len(self.get_vocab())
        return size
    
    def prob(self, context, word):
        '''
        For the probability calculation, I use the equation from the slides.
        
        You mention the novel context input, I treat it as a huge size input. 
        In my definition, I currently set it as 100, 000 words. If the size of input is more 
        than that, I treat it as a novel.
        '''
        ngram = self.update(s)
        fdist = FreqDist(context)
        context = tuple(context)
        
        v = self.size_vocab()
        
        if len(context)> 100000:
            probability = 1 / v
        elif word in fdist:
            if self.n == 1:
                probability = float(fdist[word]+1) / (v + len(ngram))
            else:
                probability = float(fdist[word]+1) / (v + ngram.count(context))
        else:
            probability = 1 / (v + 1)
        return probability
    
    def len_text(self):
        '''
        I don't know where to get the text, given the init function only requires one
        parameter, which is the n. So I put the global variable s, which is the source text
        into this function and the following one.
        ''' 
        tokenized_word = word_tokenize(s)
        return len(tokenized_word)
    
    def len_ngram(self):
        gram = self.update(s)
        return len(gram)
    
    def word_freq(self, word):
        s = s.lower()
        s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
        tokens = word_tokenize(s)
        freq = tokens.count(word) / len(tokens)
        return freq
    
    def ngram_freq(self, gram):
        ngram = self.update(s)
        freq = ngram.count(gram)
        return freq
    
    def generate_text(self, context, min_length, max_length):
        #random.seed(30)
        
        tokens = word_tokenize(context)
        token = word_tokenize(s)
        
        ngram = self.update(s)
        
        output = []
        
        locator = random.randint(1, len(ngram))
        
        if len(tokens) < self.n-1:
            input = ngram[locator]
        elif len(tokens) == self.n-1:
            context = tuple(context.split(" "))
            prefix = ' '.join(context[-(self.n-1):])
            input = '%s %s' % (prefix, tokens[0])
        else:
            context = tuple(context.split(" "))
            input = context[-(self.n-1):]

        input = ' '.join(input)
        for i in range(max_length):
            output.append(input)
            gram = ['%s %s' % (input, word) for word in ngrams]
            fdist = nltk.FreqDist(gram)
            freq_list = [(value, key) for key, value in fdist.items()]
            backup = random.choice(token)
            try:
                input = max(freq_list)
            except:
                input = backup
                continue
            
        output = ' '.join(output)
        
        return output
    
    def perplexity(self, text):
        per = 1
        num = 0
        gram = self.update(s)
        for word in text:
            if word in gram:
                num += 1
                probability = self.prob(gram, word)
                per = per * (1/probability)
        per = pow(per, 1/float(num))
    
        return per

### Question 2
> Read the given dataset from the link, create a 3-gram model for it.
> Generate a sentence of length of 30 with the given context - ‘our business’.


In [94]:
bi = NGram()
bi.n = 3    
    
output = bi.generate_text(context = 'our business', min_length = 0, max_length = 30)

print(output)

o u r   b u s i n e s s   o u r belly That : us them : if the leader it speak trusts in give must well especially as . First the not against poor but had won From in


### Question 3
> For the given context ‘make you a sword for me’, compare the perplexity of
> unigram, bigram and trigram.

The sample is a NGram object, by changing the n, you get different ngram.
I use the for-loop to show you the unigram, bigram, trigram and their results.

In [17]:
test = 'make you a sword for me'
tokens = word_tokenize(test) 

for i in range(3):
    sample = NGram()
    sample.n = i + 1
    ngrams = zip(*[tokens[j:] for j in range(i+1)])
    testgram =  [" ".join(ngram) for ngram in ngrams]
    per = sample.perplexity(testgram)
    print(i+1, "gram perplexity: ", per)

1 gram perplexity:  276.80043079892704
2 gram perplexity:  636.9446721022398
3 gram perplexity:  802.5
